## 深度学习自然语言处理第四次作业

##### 李明昕 SY2206124

### 1. 模型

本次作业使用 **LSTM** 实现一个 **生成式语言模型**。

#### 1.1 生成式语言模型

可以用单向语言模型来建模生成式语言模型。

也就是说，对于一段文本序列 $\{w_1, w_2, \cdots, w_n\}$，其概率可以表示为：
$$
P(w_1,\cdots,w_n)=\prod_{i=1}^{n}P(w_i|w_{<i})
$$

对于文本生成，根据已有的文本序列 $\{w_1,\cdots,w_k\}$，可以利用语言模型给出的 $P(w_{k+1}|w_{<k+1})$ 通过解码算法完成文本生成的过程。

常见的解码算法包括：

1. 贪心算法：在解码的过程中每次只选择概率最高的字（或词）进行生成；
2. beam-search：维护一个大小为 $m$ 的窗口，生成的过程中保留概率前 $m$ 大的生成结果；
3. 基于采样的算法：在生成的过程中不直接选择概率最高的字（或词）进行生成，而是通过概率进行采样生成。

在解码的过程中可以通过一下三个参数调整概率分布：

1. topK: 只保留分布中概率前 $K$ 大的字（或词），并对概率重新进行归一化；
2. topP: 只保留分布中概率加和刚好超过 $P$ 的前几个字（或词），并对概率进行重新归一化；
3. $\tau$: 由于分布通常是通过 softmax 计算得到的，所以可以通过温度参数 $\tau$ 改变 softmax 的结果。$\tau$ 越大分布越均匀，$\tau$ 越小分布越陡峭。

#### 1.2 LSTM

LSTM（长短期记忆网络）是一种循环神经网络（RNN）的变体，特别适用于处理具有时间依赖性的序列数据。LSTM通过引入门控机制，能够有效地解决传统RNN中的长期依赖问题。LSTM的核心思想是维护一个内部记忆单元（cell state），并通过三个门控单元（输入门、遗忘门和输出门）来控制内部记忆单元的读写和遗忘操作。这些门控单元使用sigmoid函数和逐元素乘法来决定信息的流动。

具体来说，对于时间步$t$：假设输入为$x(t)$，前一隐藏状态为$h(t-1)$，前一记忆单元状态为$c(t-1)$，则：
1. 输入门（input gate）:
   $$
   i(t) = \sigma(W(i) \cdot [h(t-1), x(t)] + b(i))
   $$
2. 遗忘门（forget gate）:
   $$
   f(t) = \sigma(W(f) \cdot [h(t-1), x(t)] + b(f))
   $$
3. 输出门（output gate）:
   $$
   o(t) = \sigma(W(o) \cdot [h(t-1), x(t)] + b(o))
   $$
4. 新的记忆单元状态（cell state）:
   $$
   c'(t) = \tanh(W(c) \cdot [h(t-1), x(t)] + b(c))
   $$
5. 当前记忆单元状态（cell state）:
   $$
   c(t) = f(t) \cdot c(t-1) + i(t) \cdot c'(t)
   $$
6. 当前隐藏状态（output）:
   $$
   h(t) = o(t) \cdot \tanh(c(t))
   $$

在上述公式中，$W$和$b$分别表示权重和偏置项。$[h(t-1), x(t)]$表示将前一隐藏状态$h(t-1)$和当前输入$x(t)$进行拼接。$\sigma$表示sigmoid函数，$\tanh$表示双曲正切函数。

### 2. 语料库

语料库一共包含了**金庸**以及**古龙**两位武侠小说作家的共 94 部小说，其中金庸的小说共 16 部，古龙的小说共 78 部。

#### 2.1 数据预处理

首先去除小说中的广告以及空白字符，然后将两位作者的小说分别整合到一个文件中。

In [1]:
import re
import os

In [2]:
DATA_DIR = 'resources'
def clean_and_collect(author: str):
    ad_p = re.compile(r"本书来自www.cr173.com免费txt小说下载站\n更多更新免费电子书请关注www.cr173.com")
    b_p = re.compile(r"\s")
    nc_p = re.compile(r"[^\u4e00-\u9fa5，…：、。！？；]")
    
    books = os.listdir(os.path.join(DATA_DIR, author))
    corpus = ''

    for book in books:
        with open(os.path.join(DATA_DIR, author, book), 'r', encoding='gb2312', errors='ignore') as fi:
            corpus += nc_p.sub('', b_p.sub('', ad_p.sub('', fi.read())))
    
    with open(os.path.join(DATA_DIR, author, 'corpus'), 'w', encoding='utf8') as fo:
        fo.write(corpus)

In [3]:
AUTHORS = ['jinyong', 'gulong']
for author in AUTHORS:
    clean_and_collect(author)

#### 2.2 分词并构建词表

考虑两种分词方法：

1. 以字为单位进行分词；
2. 通过 jieba 分词以词为单位进行分词。

两种分词单位下，分词的结果如下：

|分词单位|金庸小说 token 总数|古龙小说 token 总数 | 词汇表大小|
|---|---|---|---|
|字|8295346|16370203|5772|
|词|5314006|10783166|292996|


In [4]:
import jieba

In [5]:
def tokenize(tokenize_type: str = 'char'):
    tokenized_corpus = {}
    tokens = set()
    for author in AUTHORS:
        with open(os.path.join(DATA_DIR, author, 'corpus'), 'r', encoding='utf8') as fi:
            if tokenize_type == 'char':
                tokenize_fn = list
            else:
                tokenize_fn = lambda x: list(jieba.cut(x))
            tokenized_corpus[author] = tokenize_fn(fi.read())
            tokens.update(tokenized_corpus[author])
    token2id = {token: i for i, token in enumerate(tokens)}
    id2token = {i: token for token, i in token2id.items()}
    return tokenized_corpus, token2id, id2token

### 3. 实现

#### 3.1 基于 LSTM 的生成式语言模型

模型主要包括三个部分：

1. 嵌入层：将 token idx 所对应的 one-hot 向量转换为稠密的特征向量；
2. LSTM 层：由多层包含 LSTM 以及前馈神经网络的模块组成；
3. 输出层：将 token 对应的特征向量转换为此表上的 logits ，用于在 softmax 中计算概率分布。

In [6]:
import torch
from torch import nn
from torch.nn import functional as F

/home/bheveryday/software/miniconda3/envs/NLP/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class LSTMBlock(nn.Module):
    def __init__(self, embed_size: int, dropout: float = 0.2):
        super().__init__()
        self.lstm = nn.LSTM(embed_size, embed_size, batch_first=True)
        self.lstm_dropout = nn.Dropout(dropout)
        self.feedforward = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(),
            nn.Linear(embed_size * 4, embed_size),
            nn.Dropout(dropout)
        )
        self.l1 = nn.LayerNorm(embed_size)
        self.l2 = nn.LayerNorm(embed_size)

    def forward(self, x):
        x_res = x
        x, _ = self.lstm(self.l1(x))
        x = x_res + self.lstm_dropout(x)
        x = x + self.feedforward(self.l2(x))
        
        return x

In [8]:
class LSTMLanguangeModel(nn.Module):
    def __init__(self, vocab_size: int, embed_size: int, layer_num: int, dropout: float = 0.2):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_size)
        self.blocks = nn.Sequential(*[LSTMBlock(embed_size, dropout) for _ in range(layer_num)])
        self.lf = nn.LayerNorm(embed_size)
        self.lm_head = nn.Linear(embed_size, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        x = self.token_embedding(idx)
        x = self.blocks(x)
        x = self.lf(x)
        
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, _ = logits.shape
            logits = logits.view(B*T, -1)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

#### 3.2 模型训练

##### 3.2.1 数据加载

在训练时，同时使用金庸以及古龙的语料进行训练。对于金庸的语料，采用顺序抽取段落的方式以保证所有文本都能得到训练；对于古龙的语料，采用随机抽取段落的方式以增加训练语料的多样性。

In [9]:
import math
import random

In [10]:
class Corpus:
    def __init__(self, seq_len: int, step_interval: int, batch_size: int, tokenize_type: str = 'char', eval_p: float = 0.05):
        self.corpus, self.token2id, self.id2token = tokenize(tokenize_type)
        self.corpus['jinyong'], self.eval_corpus = self.corpus['jinyong'][:int(len(self.corpus['jinyong']) * (1 - eval_p))], \
            self.corpus['jinyong'][int(len(self.corpus['jinyong']) * (1 - eval_p)): ]

        self.batch_size = batch_size
        self.seq_len = seq_len
        self.step_interval = step_interval

    def get_train_data(self):
        batch_size_jy = int(self.batch_size * 0.7)
        batch_size_gl = self.batch_size - batch_size_jy

        jy_i = 0
        while jy_i < len(self.corpus['jinyong']) - self.seq_len:
            jy_is = list(range(jy_i, 
                         min(len(self.corpus['jinyong']) - self.seq_len, jy_i + batch_size_jy * self.step_interval),
                         self.step_interval))
            jy_i = jy_is[-1] + self.step_interval
            gl_is = [random.randint(0, len(self.corpus['gulong']) - self.seq_len) for _ in range(batch_size_gl)]

            input_ids = [[self.token2id[token] for token in self.corpus['jinyong'][left_i: left_i + self.seq_len]] 
                         for left_i in jy_is]
            target_ids = [[self.token2id[token] for token in self.corpus['jinyong'][left_i + 1: left_i + 1 + self.seq_len]] 
                          for left_i in jy_is]

            input_ids.extend([[self.token2id[token] for token in self.corpus['gulong'][left_i: left_i + self.seq_len]] 
                              for left_i in gl_is])
            target_ids.extend([[self.token2id[token] for token in self.corpus['gulong'][left_i + 1: left_i + 1 + self.seq_len]] 
                               for left_i in gl_is])
            
            yield torch.tensor(input_ids, dtype=torch.long), torch.tensor(target_ids, dtype=torch.long)

    def __len__(self):
        return math.ceil((len(self.corpus['jinyong']) - self.seq_len) // self.step_interval / int(self.batch_size * 0.7))

    def get_eval_data(self):
        return torch.tensor([self.token2id[token] for token in self.eval_corpus], dtype=torch.long)

##### 3.2.2 模型训练

In [ ]:
import time

In [11]:
import math
import random
import torch
import time

from preprocess import tokenize

class Corpus:
    def __init__(self, seq_len: int, step_interval: int, batch_size: int, tokenize_type: str = 'char', eval_p: float = 0.05):
        self.corpus, self.token2id, self.id2token = tokenize(tokenize_type)
        self.corpus['jinyong'], self.eval_corpus = self.corpus['jinyong'][:int(len(self.corpus['jinyong']) * (1 - eval_p))], \
            self.corpus['jinyong'][int(len(self.corpus['jinyong']) * (1 - eval_p)): ]

        self.batch_size = batch_size
        self.seq_len = seq_len
        self.step_interval = step_interval

    def get_train_data(self):
        batch_size_jy = int(self.batch_size * 0.7)
        batch_size_gl = self.batch_size - batch_size_jy

        jy_i = 0
        while jy_i < len(self.corpus['jinyong']) - self.seq_len:
            jy_is = list(range(jy_i, 
                         min(len(self.corpus['jinyong']) - self.seq_len, jy_i + batch_size_jy * self.step_interval),
                         self.step_interval))
            jy_i = jy_is[-1] + self.step_interval
            gl_is = [random.randint(0, len(self.corpus['gulong']) - self.seq_len) for _ in range(batch_size_gl)]

            input_ids = [[self.token2id[token] for token in self.corpus['jinyong'][left_i: left_i + self.seq_len]] 
                         for left_i in jy_is]
            target_ids = [[self.token2id[token] for token in self.corpus['jinyong'][left_i + 1: left_i + 1 + self.seq_len]] 
                          for left_i in jy_is]

            input_ids.extend([[self.token2id[token] for token in self.corpus['gulong'][left_i: left_i + self.seq_len]] 
                              for left_i in gl_is])
            target_ids.extend([[self.token2id[token] for token in self.corpus['gulong'][left_i + 1: left_i + 1 + self.seq_len]] 
                               for left_i in gl_is])
            
            yield torch.tensor(input_ids, dtype=torch.long), torch.tensor(target_ids, dtype=torch.long)

    def __len__(self):
        return math.ceil((len(self.corpus['jinyong']) - self.seq_len) // self.step_interval / int(self.batch_size * 0.7))

    def get_eval_data(self):
        return torch.tensor([self.token2id[token] for token in self.eval_corpus], dtype=torch.long)
    

class Trainer:
    def __init__(self, model, corpus, epoch, lr, wu_steps, device):

        self.device = device

        self.model = model.to(device=device)
        self.corpus = corpus
        self.step_per_epoch = len(self.corpus)
        self.epoch = epoch
        self.min_lr, self.max_lr = lr

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.max_lr)
        self.scheduler = torch.optim.lr_scheduler.LambdaLR(
            self.optimizer,
            lr_lambda=lambda cur_iter: cur_iter / wu_steps if cur_iter < wu_steps else
                (self.min_lr + 0.5*(self.max_lr - self.min_lr) * 
                 (1 + math.cos(((cur_iter - wu_steps) / (self.step_per_epoch * epoch - wu_steps) * math.pi)))) / 
                 self.max_lr
        )
        self.loss_log = []

    def train(self, check_interval=100):
        global_step = 0
        min_loss = float('inf')
        start_time = time.time()
        for epoch in range(self.epoch):
            for input_ids, target_ids in self.corpus.get_train_data():
                input_ids = input_ids.to(device=self.device)
                target_ids = target_ids.to(device=self.device)

                _, loss = self.model(input_ids, target_ids)

                loss.backward()

                self.optimizer.step()
                self.optimizer.zero_grad()
                self.scheduler.step()

                self.loss_log.append(loss.item())
                global_step += 1
                if global_step % check_interval == 0:
                    if self.loss_log[-1] < min_loss:
                        suffix = ", model saved to resources/ckpt/min_loss.ckpt"
                        torch.save(self.model.state_dict(), 'resources/ckpt/min_loss.ckpt')
                    else:
                        suffix = ''
                    step_time = (time.time() - start_time) / global_step
                    print(f'Epoch: {epoch}, GlobalStep: {global_step}, lr: {self.scheduler.get_last_lr()[0]:.5f}, eta: {step_time * (self.step_per_epoch * self.epoch - global_step) / 3600:.3f}h, loss: {self.loss_log[-1]:.4f}{suffix}')
            torch.save(self.model.state_dict(), f'resources/ckpt/{epoch}.ckpt')
            print(f'Save Model of Epoch: {epoch} to resources/ckpy/{epoch}.ckpt')  

### 4. 实验

#### 4.1 以字为单位进行分词得到的语言模型

In [12]:
corpus = Corpus(128, 16, 64, 'char', 0.05)
model = LSTMLanguangeModel(len(corpus.token2id), 384, 12, 0.2)
trainer = Trainer(model, corpus, 2, (5e-5, 1e-4), 1000, torch.device('cuda:0'))

# trainer.train(100)

: 

: 